**0. Importing libraries**

The dynamic nature of the aviation sector and the wide range of factors that impact the flight prices , it can be difficult for travelers to predict and budget for changes in airfare. It is clear that precise and trustworthy flight price forecasts are necessary to give customers the power to make wise choices. The numbers in transport through flight travel has been increasing day by day and according to transportation statistics US airlines carried more than 194 million passengers in 2022 than in 2021.The application of Data mining  techniques to predict the prices of flight tickets can be largely useful to the airline industry because of the fact that fares are subjected to wild fluctuations and varying ranges due to various factors including durations , airlines, month of travel, departure durations .

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
import joblib

**1. Data Preprocessing**

****1.1 Loading Data****

In [ ]:


df1 = r'C:\economy.csv'
df2 = r'C:\business.csv'

In [ ]:

economy_df = pd.read_csv(df1,  low_memory=False)
business_df = pd.read_csv(df2,  low_memory=False)

****1.2 Knowing the data (Economy class)****

In [ ]:
economy_df.head()

In [ ]:
economy_df.shape

Getting unique values from date column

In [ ]:
economy_df['date'].unique()

In [ ]:
economy_df = economy_df.dropna(subset=['date'])

In [ ]:
economy_df.info()

In [ ]:
economy_df['date'].unique()

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
# Convert the 'date' column to datetime with potential mixed formats and coerce errors
economy_df['date'] = pd.to_datetime(economy_df['date'], format='%d-%m-%Y', dayfirst=True, errors='coerce')

In [ ]:

target_date = datetime(2022, 2, 10)


Creating days_left column with present date 10/02/2022

In [ ]:
# Calculate the difference in days and store it in a new column 'days_left'
economy_df['days_left'] = (economy_df['date'] - target_date).dt.days


In [ ]:
# Fill NaN values with 0
economy_df['days_left'] = economy_df['days_left'].fillna(0).astype(int)

In [ ]:
economy_df['days_left'].unique()

In [ ]:
economy_df.head()

Cleaning the stop column values to one stop or two stop or more 

In [ ]:

economy_df['stop'].unique()

In [ ]:
economy_df['stop'] = economy_df['stop'].str.extract(r'(\d+)\+?-?stop')

In [ ]:
# Fill NaN values in the 'stop' column with '0'
economy_df['stop'] = economy_df['stop'].fillna('0')
# Fill NaN values in the 'stop' column with 0

In [ ]:
#df['stops'] = df['stops'].fillna('0')
economy_df['stop'].unique()

mapping to string values based on numerical for stop column

In [ ]:
stop_mapping = {0: 'zero', '0': 'zero', 1: 'one', '1': 'one', 2: 'two', '2': ' two or more'}

In [ ]:
economy_df['stop'] = economy_df['stop'].replace(stop_mapping)

In [ ]:
economy_df['stop'].unique()

Converting the time_taken  in string to travel_time column with numerical values.Since more time taken the more the cost it would be generally in the real world which can be verified later.

In [ ]:
def convert_to_float(time_str):
    parts = time_str.replace('h', ' ').replace('m', '').split()
    if len(parts) == 2:
        return round(float(parts[0]) + float(parts[1]) / 60.0, 2)
    elif len(parts) == 1:
        return round(float(parts[0]), 2)
    else:
        return None

# Apply the function to the 'time_taken' column and create a new 'time_taken_float' column
economy_df['travel_time'] = economy_df['time_taken'].apply(convert_to_float)

In [ ]:
economy_df['class']='Economy'

# Saving the DataFrame to a CSV file without index
economy_df.to_csv(r'C:\Users\Mouli\Desktop\Flightpirceprediction\Flightpirceprediction\part1.csv', index=False)


****1.3 Knowing the data (Business Class)****

In [ ]:
business_df.head()

Doing the same operations to clean the business dataset as done for the economy dataset

In [ ]:
business_df = business_df.dropna(subset=['date'])

In [ ]:
# Convert the 'date' column to datetime with potential mixed formats and coerce errors
business_df['date'] = pd.to_datetime(business_df['date'], format='%d-%m-%Y', dayfirst=True, errors='coerce')

# Specify the target date as 10-10-2023
target_date = datetime(2022, 2, 10)

# Calculate the difference in days and store it in a new column 'days_left'
business_df['days_left'] = (business_df['date'] - target_date).dt.days

In [ ]:
# Fill NaN values with 0
business_df['days_left'] = business_df['days_left'].fillna(0).astype(int)

business_df['days_left'].unique()

business_df['stop'] = business_df['stop'].str.extract(r'(\d+)\+?-?stop')

# Fill NaN values in the 'stop' column with '0'
business_df['stop'] = business_df['stop'].fillna('0')
# Fill NaN values in the 'stop' column with 0
#df['stops'] = df['stops'].fillna('0')
business_df['stop'].unique()

In [ ]:
stop_mapping = {0: 'zero', '0': 'zero', 1: 'one', '1': 'one', 2: 'two', '2': ' two or more'}
business_df['stop'] = business_df['stop'].replace(stop_mapping)

def convert_to_float(time_str):
    parts = time_str.replace('h', ' ').replace('m', '').split()
    if len(parts) == 2:
        return round(float(parts[0]) + float(parts[1]) / 60.0, 2)
    elif len(parts) == 1:
        return round(float(parts[0]), 2)
    else:
        return None

In [ ]:
# Apply the function to the 'time_taken' column and create a new 'time_taken_float' column
business_df['travel_time'] = business_df['time_taken'].apply(convert_to_float)


business_df['class']='Business'

In [ ]:
business_df.head()

In [ ]:
# Saving the DataFrame to a CSV file without index
business_df.to_csv(r'C:\Users\Mouli\Desktop\Flightpirceprediction\Flightpirceprediction\part2.csv', index=False)

****1.4 Loading preprocessed datasets****

In [ ]:
df3 = r'C:\Users\Mouli\Desktop\Flightpirceprediction\Flightpirceprediction\part1.csv'
df4 = r'C:\Users\Mouli\Desktop\Flightpirceprediction\Flightpirceprediction\part2.csv'

In [ ]:
%%time
df3 = pd.read_csv(df3,  low_memory=False)
df4 = pd.read_csv(df4,  low_memory=False)

**1.5 Merging datasets**

In [ ]:
# Merge the DataFrames based on a common column (e.g., 'key_column')
result_df = pd.concat([df3, df4], ignore_index=True)
result_df = result_df.drop(columns=['date','ch_code','num_code'])

In [ ]:
df=result_df

categoring the departure time and arrival time to ealry morning /afternoon /evening/night based on the numerical values prresent in the flight_times

In [ ]:
# Function to categorize flight times
def categorize_flight_time(time_str):
    time = int(time_str.replace(':', '').replace(':', ''))
    if 0 <= time < 700:
        return 'Early Morning'
    elif 700 <= time < 1200:
        return 'Morning'
    elif 1200 <= time < 1600:
        return 'Afternoon'
    elif 1600 <= time < 1930:
        return 'Evening'
    else:
        return 'Night'

# Apply the function to create a new 'time_category' column
df['departure_time'] = df['dep_time'].apply(categorize_flight_time)


df['arrival_time']=df['arr_time'].apply(categorize_flight_time)


Replacing the airlines data of Indian Airlines to Similar American Airlines

In [ ]:
df['airline'] = df['airline'].replace('SpiceJet', 'Spirit')

In [ ]:
replace_dict = {'AirAsia': 'Alaska', 'Indigo': 'Southwest', 'Vistara': 'Delta','GO_FIRST': 'United_Airlines','Air_India': 'Frontier'}
df['airline'] = df['airline'].replace(replace_dict)
replace_dict_cities = {'Delhi': 'Newyork', 'Mumbai': 'LasVegas', 'Bangalore': 'LosAngeles','Kolkata': 'Chicago','Hyderabad': 'Dallas','Chennai': 'Houston'}
df['source_city'] = df['from'].replace(replace_dict_cities)
df['destination_city'] = df['to'].replace(replace_dict_cities)

In [ ]:
df['price'].describe()

In [ ]:
df['price'] = df['price'].str.replace(',', '', regex=True)  # Remove commas
df['price'] = pd.to_numeric(df['price'], errors='coerce')

Converting the price in Indian Rupees to American Dollars(1 USD=80 INR approximately)

In [ ]:
df['price']=df['price']/80

In [ ]:
df['price'].head

In [ ]:
df['price'].unique()

Renaming the column names

In [ ]:
replace_dict = {'from': 'Source_City', 'to': 'Destination_City'}

In [ ]:
df = df.drop(columns=['from','to'])

Obtaining the final cleaned csv 

In [ ]:
df = r'C:\Cleaned.csv'

In [ ]:
%%time
df = pd.read_csv(df,  low_memory=False)

Findind the missing values in the cleaned dataset

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

There are no missing values in the cleaned dataset now .id there are any in any column we can drop using the command shown in the below cell.if there are null rows present in price column ,then we can drop the rows using the below command.

In [ ]:
df.shape

In [ ]:
df = df.dropna(subset=['price'])

In [ ]:
df.shape

Performing Data visualisation 

In [ ]:
fig = px.box(df, x="airline", y="price", color="class" ,color_discrete_map={ "Business": "green", "Economy": "orange"})
fig.show()

Plotting Box Whisker plot for the price variable

In [ ]:
sns.boxplot(x=df["price"])

We can observe there are some outliers that lies above the 75th percentile and we will use zscore >3 to judge whether there are really potential outliers in the price column

In [ ]:
df.price.info()

In [ ]:
df.price.describe()

In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for potential outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify potential outliers
outliers = df[(df['price'] < lower_bound) | (df['price'] > upper_bound)]

# Display the potential outliers
print("Potential Outliers:")
print(outliers)

To check for outliers in better way we implement z-score >3 and detect the number of rows which contain price outliers 

In [ ]:
from scipy.stats import zscore
df['z_score'] = zscore(df['price'])

# Identify potential outliers with Z-score greater than 3 or less than -3
potential_outliers = df[abs(df['z_score']) > 3]

# Display the potential outliers
print("Potential Outliers:")
print(potential_outliers)

602 ROWS contain price outliers here 

In [ ]:
df.shape

In [ ]:
df['z_score'] = zscore(df['price'])

# Identify potential outliers with Z-score greater than 3 or less than -3
potential_outliers = df[abs(df['z_score']) > 3]

# Drop rows with potential outliers
df = df.drop(potential_outliers.index)

Dropping rows with price outliers .The code is performed without dropping the price outliers and the same random forest regressor gave the highest accuracy but with dropping these columns gave higher accuracy than the former.df.shape

In [ ]:
print('no of rows deleted ')
300261-299659

Performing using Univariate analysis for price and travel_time

Checking the probability density plot using KDE(Kernel Density Estimate) for continous variable travel_time

In [ ]:
sns.kdeplot(data= df, x="travel_time")

In [ ]:
sns.kdeplot(data= df, x="price")

Checking the probability density plot using KDE(Kernel Density Estimate) for continous variable price and we can see majority of the value lies below 400.

Performing BiVariate analysis

Grouping the data by the 'airline' column, calculating the sum of prices for each airline, sorting the results in descending order,

In [ ]:
airline_price_data = df.groupby('airline')['price'].sum().sort_values(ascending=False)
airline_price_data
sns.barplot(x=airline_price_data.index, y=airline_price_data.values)

 KDE plot for  'price' variable.  'Price' variable will be plotted on the x-axis. The hue='airline' argument adds color based on the 'airline' variable for different airlines

In [ ]:
plt.figure(figsize=(30, 6))
sns.kdeplot(data=df, x='price', hue='airline')

In [ ]:
df_days_left = df.groupby(['days_left'])['price'].mean().reset_index()
df_days_left

Scatter plot for price vs days_left as we can see from the below plot the number of days left increases for the booking of the flight  from the current_date ,the price decreases usually

In [ ]:
sns.scatterplot(x="days_left", y="price", data=df_days_left)

We can see from the below plot the number of days left for the travel  increases the price increases and there is a spike in the lower side of the days left ,so this wont be a hard guess but not to be trained and predicted by a good model which can intrepret the complexities present in the price date.

In [ ]:
plt.plot(df_days_left['days_left'],df_days_left['price'])

Analysing the spike for each airlines by plotting between price and different airlines and number of days left =1 or 2.

In [ ]:
filtered_data = df[df['days_left'].isin([1, 2])]
sns.pointplot(data=filtered_data, x='airline', y='price', hue='days_left')

The average values of prices for departure_time and arrival_time for the airlines .

In [ ]:
grouped_data = df.groupby(['departure_time','arrival_time'])['price'].mean().sort_values(ascending=False)
grouped_data

The below plot based on the group data shows that during (Night,evening) is stastically highest and during (Night,Early Morning) it is usually lower.This can be usually correct since people find it easy to travel during  Evening and night than compared to night and early morning it is usually lower.

In [ ]:
fig = plt.figure(figsize = (8, 3))
grouped_data.plot(kind='bar')

In [ ]:
grouped_data_dep = df.groupby(['departure_time'])['price'].mean().sort_values(ascending=False)
grouped_data_dep

Plot for price and departure_time .It can be inferred that when the departing time is in the early morning ,it is less demand and during night and morning it is in more demand and expensive since lot of people prefer these timings to be comfortable

In [ ]:
grouped_data_dep.plot(kind='bar')

Plot for price and departure_time .It can be inferred that the business class is almost same higher at all timings but economy class can vary during various times nd reach higher prices during evening  and morning times with denser numbers from the blox plot.

In [ ]:
fig = px.box(df, x="departure_time", y="price", color="class")
fig.show()

In [ ]:
grouped_data_arr = df.groupby(['arrival_time'])['price'].mean().sort_values(ascending=False)
grouped_data_arr

Plot for price and arrival_time .It can be inferred that when the arrival time is in the early morning ,it is less demand and during night and evening  it is in more demand and expensive since lot of people prefer these timings to be comfortable.Therefore customers tend to travel and arrive during nights and evenings rather than early mornings

In [ ]:
grouped_data_arr.plot(kind='bar')

Similar to departure_time this trend of business class is almost similar in average price in all timings where as economy class tends to variate and reach more denser and higher during night arrivals from the box plot.

In [ ]:
fig = px.box(df, x="arrival_time", y="price", color="class")
fig.show()

In [ ]:
grouped_data_city = df.groupby(['source_city','destination_city'])['price'].mean().sort_values(ascending=False)
grouped_data_city

From the below bar graph ,the prices between certain routes like (Hosuton,LA),(Chicago,Houston) are more expensive and (Dallas and NY ) are less expensive and might be less busier or less in demand on average.

In [ ]:
fig = plt.figure(figsize = (8, 3))
grouped_data_city.plot(kind='bar')

In [ ]:
grouped_data_src = df.groupby(['source_city'])['price'].mean().sort_values(ascending=False)
grouped_data_src


The plot indicates that Flights with certain source_destinations like Hosuton,Chicago  are  slightly higher  than Newyork on average

In [ ]:
fig = plt.figure(figsize = (8, 3))
grouped_data_src.plot(kind='bar')

In [ ]:
grouped_data_dest = df.groupby(['destination_city'])['price'].mean().sort_values(ascending=False)
grouped_data_dest

And the similar trend for flights for destination_city and average prices.

In [ ]:
fig = plt.figure(figsize = (8, 3))
grouped_data_dest.plot(kind='bar')

In [ ]:
grouped_data_city_dur = df.groupby(['source_city','destination_city'])['travel_time'].mean().sort_values(ascending=False)
grouped_data_city_dur

This is plot between source,destination cities and travel time and from the plot we can infer the travel time between certain cities is slightly higher  than (LA,NY) may be owing to intentional layovers .

In [ ]:
fig = plt.figure(figsize = (8, 3))
grouped_data_city_dur.plot(kind='bar')

The below plot gives the  average price variaton for business and economy class with business class leading to higher prices which is true generally.

In [ ]:
grouped_data_class = df.groupby('class')['price'].mean()

plt.bar(grouped_data_class.index, grouped_data_class)

This contains different airlines providing flights with different average prices with economy and business classess.

In [ ]:
sns.barplot(x='airline',y='price',hue="class", data =df)

In [ ]:
average_price_by_stops = df.groupby('stop')['price'].mean().reset_index()
average_price_by_stops

PLot for layovers and  average price and if the number of stops is one then the prices will be higher and less with zero stops and surprisingly little lesser with two or more stops when comapred with one stop.

In [ ]:
sns.barplot(data=average_price_by_stops, x='stop', y='price')

with two or more stops when comapred with one stop sometimes the average prices seems to reach higher which is usually  higher in outside world

In [ ]:
sns.boxplot(x='stop',y='price',data=df,palette='viridis')

In [ ]:
sns.kdeplot(data=df,x= 'price', hue='stop')

In [ ]:
airline_flights_price= df.groupby(['airline','source_city','destination_city'])['price'].mean().sort_values(ascending=False).reset_index()
airline_flights_price= pd.DataFrame(airline_flights_price)
airline_flights_price

In [ ]:
flightdf=df

In [ ]:
flightdf_temp = flightdf.groupby(['days_left'])['price'].mean().reset_index()
plt.figure(figsize=(15,6)).suptitle('Airline prices based on days left before buying the ticket', fontsize=16)
ax = plt.axes()
sns.regplot(x=flightdf_temp.loc[flightdf_temp["days_left"]==1].days_left, y=flightdf_temp.loc[flightdf_temp["days_left"]==1].price, fit_reg=False, ax=ax)
sns.regplot(x=flightdf_temp.loc[(flightdf_temp["days_left"]>1)&(flightdf_temp["days_left"]<20)].days_left, y=flightdf_temp.loc[(flightdf_temp["days_left"]>1)&(flightdf_temp["days_left"]<20)].price, fit_reg=True, ax=ax)
sns.regplot(x=flightdf_temp.loc[flightdf_temp["days_left"]>=20].days_left, y=flightdf_temp.loc[flightdf_temp["days_left"]>=20].price, fit_reg=True, ax=ax)

Plot for no of days_left and price and follows the similar trend as explained before 

In [ ]:
plt.figure(figsize = (18,5))
plt.subplot(1,2,1)
sns.boxplot(data=flightdf, y="price", x="departure_time",showfliers=False).set_title("Airline prices based on the departure time",fontsize=15)
plt.subplot(1,2,2)
sns.boxplot(data=flightdf, y="price", x="arrival_time",showfliers=False).set_title("Airline prices based on the arrival time",fontsize=15)

In [ ]:
ax = sns.relplot(col="source_city", y="price", kind="line",x='destination_city', data=flightdf, col_wrap=3)
ax.fig.subplots_adjust(top=0.9) # adjust the Figure in rp
ax.fig.suptitle('Airline prices based on the source and destination cities',fontsize=20)

The above plots indicating the trends as explained before in the bar graphs

In [ ]:
fig, axs = plt.subplots (1, 2, gridspec_kw={'width_ratios': [5, 3]}, figsize=(25, 5))
sns.barplot(y = "price", x = "airline",hue="stop",data = flightdf.loc[flightdf["class"]=='Economy'].sort_values("price", ascending = False), ax=axs[0])
axs[0].set_title("Airline prices based on the number of stops  for economy",fontsize=20)
sns.barplot(y = "price", x = "airline",hue="stop",data = flightdf.loc[flightdf["class"]=='Business'].sort_values("price", ascending = False), ax=axs[1])
axs[1].set_title("Airline prices based on the number of stops  for business",fontsize=20)

In [ ]:
flightdf.info()

Performing label encoding for categorical values for further evaluation and feature selection

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
flightdf.info()

In [ ]:

cat_cols =['airline','source_city','departure_time', 'stop','arrival_time','destination_city','class']

# Copy the DataFrame
finaldf = flightdf.copy()
flightdf_orginal = flightdf.copy()

In [ ]:

# Apply label encoding to the specified columns
flightdf[cat_cols] = flightdf[cat_cols].apply(le.fit_transform)


# Display unique encoded values for each categorical column for Data Preprocessing in the Model deployment stage.
for column in cat_cols:
    unique_encoded_values = flightdf[column].unique()
    print(f"Unique encoded values for column '{column}' {flightdf_orginal[column].unique()} : {unique_encoded_values}")

The categorical values are now encoded to numerics

In [ ]:
flightdf.info()

Dropping dep_time and arr_time since their aliases are already present as depature_time and arrival_time for analysis

In [ ]:
df.drop('dep_time', axis=1, inplace=True)

In [ ]:
df.drop('arr_time', axis=1, inplace=True)

In [ ]:
flightdf.info()

In [ ]:
df.drop('time_taken', axis=1, inplace=True)

Dropped the time_taken column since travel_time column is present which actually mean the same thing.

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(flightdf.corr(), annot = True, fmt=".2f")
plt.show()

Plotting a corrleation matrix and we can infer the variables class has negative corelation since we gave the business class as zero in the encoding and economy class as one in the encoding.The travel_time has positive correlation and no of stops also has negative corelation and certain destination and arrival cities have positive corelation due to values they got in encoding

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Split the dataset into features (X) and the target variable (Y)
Y = flightdf["price"]
X = flightdf.drop(["price"], axis=1)

# Feature selection using F-statistic (ANOVA)
num_features_to_select = 9  # Number of features to select
selected_features = SelectKBest(f_regression, k=num_features_to_select).fit(X, Y)

# Get the indices of the selected features
selected_feature_indices = selected_features.get_support(indices=True)

# Display the selected features and their indices

selected_feature_names = X.columns[selected_feature_indices]
print("Selected features:", selected_feature_names)

# Selected features' data
selected_data = X[selected_feature_names]
print(selected_data.head())

 perform feature selection using ANOVA(Analysis of variance between target variable price and other columns present in the dataset and select top 9 columms in the dataset since out of 12 columns (13th column is price -target variable) , one  is redundant column since travel time or travel duration both are same and dep_time  and arr_time are categorical aliases for the depature_time and arrival_time.

In [ ]:
features_to_drop = [col for col in flightdf.columns if col not in selected_feature_names and col != 'price']
flightdf.drop(features_to_drop, axis=1, inplace=True)
flightdf

dropping the departure and arrival time since they less variance from the above with respect to price

In [ ]:
flightdf = flightdf.drop('departure_time', axis=1)
flightdf = flightdf.drop('arrival_time', axis=1)

In [ ]:
flightdf

In [ ]:
final = flightdf.copy()
final

In [ ]:
import statsmodels.api as sm
Y = flightdf["price"]
X = flightdf.drop(["price"], axis=1)
#Adding constant column of ones, mandatory for sm.OLS model
X_1 = sm.add_constant(X)
#Fitting sm.OLS model
model = sm.OLS(Y,X_1).fit()

In [ ]:
cols = list(X.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = X[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(Y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

Normalising the values using minmax scaler so that other columns with higher maginitudeof numeric wont dominate the lesser ones

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Custom new minimum and maximum values
new_min = 0
new_max = 1  # Set your desired maximum value

duration_min = flightdf['travel_time'].min()
duration_max = flightdf['travel_time'].max()
days_left_min = flightdf['days_left'].min()
days_left_max = flightdf['days_left'].max()
price_min = flightdf['price'].min()
price_max = flightdf['price'].max()

In [ ]:
scaler = MinMaxScaler(feature_range=(new_min, new_max))

 # List of numerical columns to scale
num_cols = ['travel_time', 'days_left', 'price']

 # Apply min-max scaling to the specified columns in flightdf
flightdf[num_cols] = scaler.fit_transform(flightdf[num_cols])

 # Display the scaled DataFrame
flightdf

In [ ]:
Y = flightdf["price"]
X = flightdf.drop(["price"], axis=1)

 split  80 % of the dataset for the training purpose and 20 % for the training purpose and random state as ‘7’ just to ensure if we run the code again from random state we get the same split.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, test_size = 0.2, random_state = 7)

#### **Linear regression**

Performing the linear regression training and predicting

In [ ]:
from sklearn.linear_model import LinearRegression

lrm = LinearRegression()

lrm.fit(X_train,Y_train) #training the algorithm

Y_pred = lrm.predict(X_test)

retrieve the intercept and coefficients (slopes) of a linear regression model

In [ ]:
print(lrm.intercept_)

#For retrieving the slope:
print(lrm.coef_)

In [ ]:
df_lrm = pd.DataFrame({'Actual': Y_test.to_numpy().flatten(), 'Predicted': Y_pred.flatten()})

Evaluation metrics for Linear Regression

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error ,r2_score

# MSE

MSE_LR=mean_squared_error(Y_test, Y_pred)
print("MSE: ", mean_squared_error(Y_test, Y_pred))

# RMSE

RMSE_LR=np.sqrt(mean_squared_error(Y_test, Y_pred))
print("RMSE: ", np.sqrt(mean_squared_error(Y_test, Y_pred)))

# MAE

In [ ]:
MAE_LR=np.sqrt(mean_absolute_error(Y_test, Y_pred))
print("MAE: ", np.sqrt(mean_absolute_error(Y_test, Y_pred)))

# R2 Score

R2_LR=r2_score(Y_test, Y_pred)
print("R2 Score: ", r2_score(Y_test, Y_pred))

#### **Decision Tree**

In [ ]:
from sklearn import datasets

from sklearn.tree import DecisionTreeRegressor    # Import decision tree regression model

 Initialize and fit regressor,The criterion parameter specifies the function that the decision tree algorithm will use to measure the quality of a split during the tree-building process. In this case, 'mse' stands for Mean Squared Error, which is a common criterion for regression problems. It measures the average squared difference between the actual target values and the predicted values. The decision tree algorithm tries to minimize this error when splitting the data.

In [ ]:
Rtree = DecisionTreeRegressor(criterion='friedman_mse',     
                             max_depth=3)

In [ ]:
RT_model = Rtree.fit(X_train, Y_train)

Y_pred = RT_model.predict(X_test)

In [ ]:
df_RT = pd.DataFrame({'Actual': Y_test.to_numpy().flatten(), 'Predicted': Y_pred.flatten()})

df1 = df_RT.head(25)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.1')
plt.grid(which='minor', linestyle=':', linewidth='0.1')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error ,r2_score

# MSE

MSE_DTR=mean_squared_error(Y_test, Y_pred)
print("MSE: ", mean_squared_error(Y_test, Y_pred))

# RMSE

RMSE_DTR=np.sqrt(mean_squared_error(Y_test, Y_pred))
print("RMSE: ", np.sqrt(mean_squared_error(Y_test, Y_pred)))

# MAE
MAE_DTR=np.sqrt(mean_absolute_error(Y_test, Y_pred))
print("MAE: ", np.sqrt(mean_absolute_error(Y_test, Y_pred)))

# R2 Score

R2_DTR=r2_score(Y_test, Y_pred)
print("R2 Score: ", r2_score(Y_test, Y_pred))


#### **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr= RandomForestRegressor(n_estimators= 50, random_state= 0)
rfr.fit(X_train,Y_train)
Y_pred = rfr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error ,r2_score

# MSE

MSE_RFR=mean_squared_error(Y_test, Y_pred)
print("MSE: ", mean_squared_error(Y_test, Y_pred))

# RMSE

RMSE_RFR=np.sqrt(mean_squared_error(Y_test, Y_pred))
print("RMSE: ", np.sqrt(mean_squared_error(Y_test, Y_pred)))
MAE_RFR=np.sqrt(mean_absolute_error(Y_test, Y_pred))
print("MAE: ", np.sqrt(mean_absolute_error(Y_test, Y_pred)))

# R2 Score

R2_RFR=r2_score(Y_test, Y_pred)
print("R2 Score: ", r2_score(Y_test, Y_pred))


#### **XGBoost regressor**

In [ ]:
import xgboost
from xgboost import XGBRegressor

# Create an XGBRegressor model
XGB = XGBRegressor(n_estimators=100, learning_rate=0.1)
XGB.fit(X_train,Y_train)
Y_pred = XGB.predict(X_test)

In [ ]:
SE_XGB=mean_squared_error(Y_test, Y_pred)
print("MSE: ", mean_squared_error(Y_test, Y_pred))

# RMSE

RMSE_XGB=np.sqrt(mean_squared_error(Y_test, Y_pred))
print("RMSE: ", np.sqrt(mean_squared_error(Y_test, Y_pred)))

# MAE

MAE_XGB=np.sqrt(mean_absolute_error(Y_test, Y_pred))
print("MAE: ", np.sqrt(mean_absolute_error(Y_test, Y_pred)))

In [ ]:
R2_XGB=r2_score(Y_test, Y_pred)
print("R2 Score: ", r2_score(Y_test, Y_pred))

#### **Comparision Table**

In [ ]:
import pandas as pd

models = ["Linear Regression", "Decision Tree Regressor", "Random Forest Regressor", "XGBRegressor"]
MSE = [MSE_LR, MSE_DTR, MSE_RFR, RMSE_XGB]
RMSE = [RMSE_LR, RMSE_DTR, RMSE_RFR, RMSE_XGB]
MAE = [MAE_LR, MAE_DTR, MAE_RFR, MAE_XGB]
R2 = [R2_LR, R2_DTR, R2_RFR, R2_XGB]

# Create a DataFrame to display the results with indexing starting from 1 and labeled as 'Model no.'
results = pd.DataFrame({
    'Model': models,
    'MSE': MSE,
    'RMSE': RMSE,
    'MAE': MAE,
    'R Squared': R2
}, index=range(1, len(models) + 1))  # Start indexing from 1

# Rename the index column to 'Model no.'
results.index.name = 'Model no.'


def highlight_min_max(val):
    style = ''
    if isinstance(val, (int, float)):
        if val == results['MSE'].min():
            style += 'background-color: rgba(255, 0, 0, 0.3); color: black; font-weight: bold;'  # Red for minimum MSE
        if val == results['RMSE'].min():
            style += 'background-color: rgba(0, 255, 0, 0.3); color: black; font-weight: bold;'  # Green for minimum RMSE
        if val == results['MAE'].min():
            style += 'background-color: rgba(0, 0, 255, 0.3); color: black; font-weight: bold;'  # Blue for minimum MAE
        if val == results['R Squared'].max():
            style += 'background-color: rgba(255, 255, 0, 0.3); color: black; font-weight: bold;'  # Yellow for maximum R Squared
    return style


# Apply the highlight_min_max function to the entire DataFrame
styled_results = results.style.applymap(highlight_min_max)

# Display the styled results DataFrame
styled_results



**Saving Model**

The Random Forest Regressor emerges as the most effective model for flight price prediction, demonstrating superior accuracy and robustness in capturing the complexities of pricing dynamics.
Random Forest Regressor stands out as the most effective model, striking a balance between accuracy and interpretability.
XGBRegressor, while competitive, demonstrates a slightly higher error rate, suggesting that Random Forest is more suitable for this specific problem.


In [ ]:
import joblib
# Separate features and target variable
X = final.drop('price', axis=1)  # Features
y = final['price']  # Target variable

# Train Random Forest Regressor
rf_regressor = RandomForestRegressor()
rf_regressor.fit(X, y)

# Save the trained model using joblib
joblib.dump(rf_regressor, r'C:\Users\rp225995\Documents\Model.pkl')

deploying the random forest model to a pickle file

## **GUI**

creating a GUI

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import joblib

# Encoded values for categorical columns
encoded_values = {
    'Airline': {'Spirit': 5, 'Alaska': 1, 'Delta': 2, 'GO FIRST': 3, 'Southwest': 4, 'Air India': 0, 'Trujet': 7, 'StarAir': 6},
    'Stop': {'zero': 2, 'one': 1, ' two or more': 0},
    'Class': {'Economy': 1, 'Business': 0},
    'source_city': {'Newyork': 5, 'LasVegas': 3, 'LosAngeles': 4, 'Chicago': 0, 'Dallas': 1, 'Houston': 2},
    'destination_city': {'LasVegas': 3, 'LosAngeles': 4, 'Chicago': 0, 'Dallas': 1, 'Houston': 2, 'Newyork': 5}
    # Other fields are considered as normal fields
}

# Function to predict price based on user input
def predict_price():
    try:
        # Get user inputs for encoded fields
        source_city = encoded_values['source_city'].get(source_city_dropdown.get(), source_city_dropdown.get())
        destination_city = encoded_values['destination_city'].get(destination_city_dropdown.get(), destination_city_dropdown.get())
        airline = encoded_values['Airline'].get(airline_dropdown.get(), airline_dropdown.get())
        stop = encoded_values['Stop'].get(stop_dropdown.get(), stop_dropdown.get())
        class_ = encoded_values['Class'].get(class_dropdown.get(), class_dropdown.get())
        

        # Get user inputs for numerical fields
        days_left = float(days_left_entry.get()) if days_left_entry.get() else None
        travel_time = float(travel_time_entry.get()) if travel_time_entry.get() else None
        
        # Check if source city and destination city are the same
        if source_city == destination_city:
            raise ValueError("Source city and destination city cannot be the same.")
        
        # Create a feature vector based on user inputs
        user_data = [[airline, stop, days_left, travel_time, class_, source_city, destination_city]]
        
        # Make prediction using the loaded model
        predicted_price = model.predict(user_data)
        
        # Display the predicted price
        result_label.config(text=f"Predicted Price: ${predicted_price[0]:.2f}")
    
    except ValueError as ve:
        messagebox.showerror("Error", f"ValueError: {ve}. Please enter valid input.")
    except Exception as e:
        messagebox.showerror("Error", f"Prediction error: {e}")


# Load the trained model
model = joblib.load(r'C:\Users\rp225995\Documents\Model.pkl')

# Create a GUI window
root = tk.Tk()
root.title("Flight Price Prediction")

# Create a main frame
main_frame = ttk.Frame(root, padding=20)
main_frame.pack()

# Create frames for categorical and numerical inputs
categorical_frame = ttk.LabelFrame(main_frame, text="Categorical Inputs")
categorical_frame.grid(row=0, column=0, padx=10, pady=5)

numerical_frame = ttk.LabelFrame(main_frame, text="Numerical Inputs")
numerical_frame.grid(row=0, column=1, padx=10, pady=5)

# Create dropdown menus for encoded categorical columns
dropdowns = {}
for i, (column, values) in enumerate(encoded_values.items()):
    if column not in ['Arrival Time']:
        label = ttk.Label(categorical_frame, text=f'{column}:')
        label.grid(row=i, column=0, padx=5, pady=2, sticky='w')
        dropdown_var = tk.StringVar(root)
        dropdown_var.set('')  # Empty value initially
        dropdown = ttk.Combobox(categorical_frame, textvariable=dropdown_var, values=list(values.keys()), state='readonly')
        dropdown.grid(row=i, column=1, padx=5, pady=2)
        dropdowns[column] = dropdown_var

# Assign specific dropdowns to respective variables
source_city_dropdown = dropdowns['source_city']
destination_city_dropdown = dropdowns['destination_city']
airline_dropdown = dropdowns['Airline']
stop_dropdown = dropdowns['Stop']
class_dropdown = dropdowns['Class']

# Numerical input fields initially empty
days_left_label = ttk.Label(numerical_frame, text='Days Left:')
days_left_label.grid(row=0, column=0, padx=5, pady=2, sticky='w')
days_left_entry = ttk.Entry(numerical_frame)
days_left_entry.grid(row=0, column=1, padx=5, pady=2)

travel_time_label = ttk.Label(numerical_frame, text='Travel Time:')
travel_time_label.grid(row=1, column=0, padx=5, pady=2, sticky='w')
travel_time_entry = ttk.Entry(numerical_frame)
travel_time_entry.grid(row=1, column=1, padx=5, pady=2)

# Function to set numerical input fields based on dropdown selection
def set_numerical_entries(event):
    selected_airline = airline_dropdown.get()
    if selected_airline:
        days_left_entry.delete(0, tk.END)
        travel_time_entry.delete(0, tk.END)
        # Assign values based on selected dropdown value
        days_left_entry.insert(0, '5')  # Example default value for days_left
        travel_time_entry.insert(0, '10')  # Example default value for travel_time

# Bind the dropdowns to call set_numerical_entries on selection
airline_dropdown = dropdowns['Airline']
airline_dropdown.trace('w', set_numerical_entries)

# Create a button to predict
predict_button = ttk.Button(main_frame, text="Predict Price", command=predict_price)
predict_button.grid(row=1, column=0, columnspan=2, pady=10)

# Create a label to display the result
result_label = ttk.Label(main_frame, text="")
result_label.grid(row=2, column=0, columnspan=2, pady=5)

# Function to clear all input fields and output label
def clear_entries():
    for dropdown_var in dropdowns.values():
        dropdown_var.set('')
    days_left_entry.delete(0, tk.END)
    travel_time_entry.delete(0, tk.END)
    result_label.config(text="")

# Create a button to clear entries
clear_button = ttk.Button(main_frame, text="Clear", command=clear_entries)
clear_button.grid(row=3, column=0, columnspan=2, pady=10)

# Run the main loop
root.mainloop()
